In [ ]:
'''
Import the gensim library to work with word vectors
The as api import allows us to easily download pre-trained models
KeyedVectors is used to handle the word vectors efficiently 
'''
from gensim.models import KeyedVectors
import gensim.downloader as api

In [6]:
# List all available models from the gensim downloader
print("Available models:\n")
for model_name in api.info()['models'].keys():
    print(model_name)

Available models:

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
# Load the pre-trained word vectors from a binary file
# The file 'GoogleNews-vectors-negative300.bin' contains word vectors trained on Google News data

model = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


In [15]:
# Looking up a word in the model and identifying similar words 

similar_words = model.most_similar('knife')
print("\nMost similar words to 'knife':\n")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")  


Most similar words to 'knife':

fork: 0.8200
sword: 0.7996
rope: 0.7915
blade: 0.7884
hammer: 0.7757
spoon: 0.7756
bullet: 0.7733
arm: 0.7698
hole: 0.7667
knives: 0.7633


In [ ]:
# Looking for analogous words.

analogous = model.most_similar(positive=["king", "woman"], negative=["man"])
for word, analogy in analogous:
    print(f"{word}: {analogy:.4f}")  

[('prince', 0.7597796320915222), ('stone', 0.7595877051353455), ('queen', 0.7546268701553345), ('meets', 0.7404001951217651), ('royal', 0.7376409769058228), ('african', 0.7345032095909119), ('american', 0.7139527201652527), ('virgin', 0.7101486921310425), ('mother', 0.7023203372955322), ('called', 0.6990438103675842)]
prince: 0.7598
stone: 0.7596
queen: 0.7546
meets: 0.7404
royal: 0.7376
african: 0.7345
american: 0.7140
virgin: 0.7101
mother: 0.7023
called: 0.6990


In [23]:
first_word = analogous[0][0]
print (f"\nAnalogous words:  'king' is to 'man' as 'queen' is to '{first_word}'.")


Analogous words:  'king' is to 'man' as 'queen' is to 'prince'.
